In [ ]:
!pip install --user -r requirements.txt

In [16]:
import pandas as pd
import numpy as np
import requests
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.keys import Keys
import datetime as dt
import time
from random import randint
from time import sleep
from collections import Counter

# Function for scraping the cbonds website

In [17]:
def scrape_correct_names(isin_list):
    for isin in isin_list:
        
        search_bar = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="mainSearch"]')))
        
        # click search bar
        search_bar.click()
        
        # random sleep time between 1 and 3 seconds
        sleep(randint(1,3))
        
        # select all text in the search bar and delete
        search_bar.send_keys(Keys.COMMAND, 'a')
        search_bar.send_keys(Keys.BACKSPACE)
        
        # random sleep time between 1 and 3 seconds
        sleep(randint(1,3))
        
        # type in isin number
        search_bar.send_keys(isin)
        
        try:
            # click the selection
            pop_down = wait.until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[2]/div[1]/div[1]/div[4]/div/ul[2]/li/a')))
            pop_down.click()                                                 
        
            # grab info from isin entry pop down
            title = wait.until(EC.presence_of_element_located((By.XPATH,'/html/body/div[2]/div[4]/div[2]/div[1]/h1')))
            bond_info = driver.find_element_by_xpath('/html/body/div[2]/div[4]/div[2]/div[1]/p')
            amount = driver.find_element_by_class_name('other_param')
            bond_title = title.text  
            bond_type = bond_info.text
            bond_amount = amount.text
            bond_titles.append(bond_title)
            bond_types.append(bond_type)
            bond_amounts.append(bond_amount)
          
            # random sleep time between 2 and 4 seconds
            sleep(randint(2,4))
            # issuer info
            issue_info = driver.find_element_by_id('bondInfo')
            # move to section
            driver.execute_script("arguments[0].scrollIntoView();", issue_info)
            bottom_container = issue_info.text
            issue_info_list.append(bottom_container)
    #         print(bottom_container)
            print("all info grabbed for this bond")
            # sleep 3 seconds
            time.sleep(3)
            # move back up to search bar
            back_to_search_bar = driver.find_element_by_id('mainSearch')
            back_to_search_bar.location_once_scrolled_into_view
        
        except TimeoutException:
            not_found = "isin_code_not_found"
            bond_titles.append(not_found)
            bond_types.append(not_found)
            bond_amounts.append(not_found)
            issue_info_list.append(not_found)
            continue
        sleep(randint(2,4))

    print('done')

# Import neccesary datasets

In [18]:
# import the contents of CSPP_PEPP_corporate_bond_holdings_20210430 into a dataframe
path = 'https://raw.githubusercontent.com/gabriellavinco/ECB-Bond-Purchases/main/Datasets/CSPP_PEPP_corporate_bond_holdings_20210430.csv'
corporate_bond_holdings_df = pd.read_csv(path, sep=';', index_col=None, header=0)

In [19]:
# import cbonds list to get all the isin codes used in the historical datasets
cbonds  = pd.read_csv('https://www.dropbox.com/s/3i90x0s8vt8bxpl/ALL_HISTORICAL.csv?dl=1')

In [20]:
cbonds = cbonds.dropna()
cbonds["ISIN_Code"] = cbonds["ISIN_Code"].astype(str)
cbonds

,quarter,year,ISIN_Code
0,Q2,2017,XS1088274169
1,Q2,2017,XS1088274672
2,Q2,2017,XS1144492532
3,Q2,2017,XS1571982468
4,Q2,2017,XS0859920406
...,...,...,...
2081,Q2,2021,FR0013448032
2082,Q2,2021,XS2337060607
2083,Q2,2021,XS2337061670
2084,Q2,2021,XS2337061753


In [21]:
na_check2 = cbonds.isna().any()
na_check2

quarter      False
year         False
ISIN_Code    False
dtype: bool

In [22]:
# make list of ISIN numbers
isin_list = cbonds["ISIN_Code"].tolist()

In [23]:
cbonds_quarter = cbonds["quarter"].tolist()

In [24]:
cbonds_year = cbonds["year"].tolist()

# Use cbonds.com to get info about each of the bonds by ISIN code

### Here we used selenium on the cbonds website to scrape the available data. We decided to use this method because cbonds had sufficient information for the analyses that we wanted to perform as well as information that was accessable by ISIN code lookup.

In [52]:
# URL for the webpage that we want to scrape
cbonds_url = 'https://cbonds.com'

In [ ]:
# Install and initiate driver
driver = webdriver.Chrome(ChromeDriverManager().install())
# initiate the driver
driver.get(cbonds_url)
# initiate the action object
action = webdriver.ActionChains(driver)
# create a wait with a timeout of 30 seconds
wait = WebDriverWait(driver, 30)

In [560]:
# create lists to append items from the function into 
bond_titles = []
bond_types = []
bond_amounts = []
issue_info_list = []

# Running script for the main dataset

In [ ]:
# JUST ADDED BY GAB
scrape_correct_names(isin_list)

In [481]:
len(bond_titles)

2085

In [513]:
# set to string so we dont get the .0 after year
cbonds["year"] = cbonds["year"].astype(str)

In [514]:
# add everything to a dataframe so I dont accidentally lose it
cbonds_info_df = pd.DataFrame(bond_titles)
cbonds_info_df['bond_types'] = bond_types
cbonds_info_df['bond_amounts'] = bond_amounts
cbonds_info_df['issue_info'] = issue_info_list
cbonds_info_df['ISIN_Code'] = cbonds["ISIN_Code"]
cbonds_info_df['quarter'] = cbonds["quarter"]
cbonds_info_df['year'] = cbonds["year"]

In [ ]:
# df check
cbonds_info_df.head()

In [516]:
# save the unique_improved to csv file
# cbonds_info_df.to_csv (r'/Users/gabbyvinco/Desktop/may15cbonds.csv', index = False, header=True)

In [25]:
# import cbonds_info_df list (only for debugging so the entire list doesn't have to be run again)
# cbds_info_path = r'/Users/gabbyvinco/Desktop/may15cbonds.csv'
# cbonds_info_df = pd.read_csv(cbds_info_path, index_col=None, header=0)

In [26]:
cbonds_info_df["Quarter"] = cbonds_quarter
cbonds_info_df["Year"] = cbonds_year

In [27]:
# create a df of items that isin_code was not found on cbonds
isin_codes_not_found = cbonds_info_df.loc[cbonds_info_df['bond_types'] == "isin_code_not_found"]
len(isin_codes_not_found)

11

In [820]:
# here we save the unique_improved to csv file so that we can attempt to re-run the missing isin codes 
# rather than re-runnning the entire script

# isin_codes_not_found.to_csv (r'/Users/gabbyvinco/Desktop/missing_isin_codes.csv', index = False, header=True)

In [28]:
cbonds_info_df = cbonds_info_df[cbonds_info_df.bond_types != "isin_code_not_found"]

In [29]:
cbonds_info_df = cbonds_info_df.drop(columns=['ISIN_Code', 'quarter', 'year'])

In [30]:
# check for nan values
na_check1 = cbonds_info_df.isna().any()
na_check1

0               False
bond_types       True
bond_amounts    False
issue_info      False
Quarter         False
Year            False
dtype: bool

In [31]:
# reset index
cbonds_info_df = cbonds_info_df.reset_index(drop=True)

In [791]:
# exit from the driver
driver.quit()

In [32]:
# just to see the different combinations of bond types
cbonds_info_df["bond_types"].value_counts()

Senior Unsecured                                    919
Green bonds, Senior Unsecured                        78
Zero-coupon bonds, Senior Unsecured                  45
Zero-coupon bonds                                    38
Floating rate                                        37
Green bonds                                          32
Senior Secured                                       21
Senior Unsecured, Floating rate                      11
Foreign bonds                                         6
Green bonds, Zero-coupon bonds, Senior Unsecured      4
Foreign bonds, Senior Unsecured                       2
Green bonds, Senior Secured                           2
Zero-coupon bonds, Senior Secured                     2
Foreign bonds, Floating rate                          1
Securitization                                        1
Name: bond_types, dtype: int64

In [33]:
# change bond_types column to list
b_types = cbonds_info_df["bond_types"]
b_types = b_types.tolist()

# change bond_amounts column to list
extra_info = cbonds_info_df["bond_amounts"]
extra_info = extra_info.tolist()

# change the first column to list
basic_info = cbonds_info_df["0"]
basic_info = basic_info.tolist()

# change issue_info to list
issue_info = cbonds_info_df['issue_info']
issue_info = issue_info.tolist()


In [34]:
# create new lists to split items into from extra_info
amount_list = []
status_list = []
country_of_risk_list = []

In [35]:
# split up the items of the bond_amounts column
for item in extra_info:
    item = item.split("\n")
#     print(len(item))
    amount = item[3]
    status = item[1]
    country_of_risk = item[9]
    amount_list.append(amount)
    status_list.append(status)
    country_of_risk_list.append(country_of_risk)

In [36]:
# create lists to split items into from basic_info
bond_location_list = []
name_list = []
percentage_list = []
date_list = []
currency_list = []
isin_code_list = []

In [37]:
# split up the data from the first column into its individual variables
for item in basic_info:
    item = item.split(",")
    bondNname = item[0]
    percNdate = item[1]
    if len(item) == 3:
        ctryNcode = item[2]
        ctryNcode = ctryNcode.split("\n")
        currency = ctryNcode[0]
    #     print(currency)
        isin_code = ctryNcode[1]
    else:
        isin_code = "none"
        currency = "none"
    currency_list.append(currency)
    isin_code_list.append(isin_code)
        
    bondNname = bondNname.split(": ")
    bond_location = bondNname[0]
    name = bondNname[1]
    bond_location_list.append(bond_location)
    name_list.append(name)
    
    percNdate = percNdate.split(" ")
    if len(percNdate) == 2:
        sub_item = percNdate[1]
        sub_item = sub_item.split("%")
        date = sub_item[1]
        percentage = sub_item[0]+"%"
        percentage_list.append(percentage)
        date_list.append(date)
    else:
        percentage = percNdate[1]
        date = percNdate[2]
        percentage_list.append(percentage)
        date_list.append(date)
    
    
    

In [38]:
# create lists to split items into from issue_info
comp_text = []
sector = []
industry = []
nominal = []
stocks = []
relationship_status = []

In [39]:
for item in issue_info:
    sector_split = item.split("Sector\n")
    sector_info = sector_split[1]
    sector_info = sector_info.split("\n")
    sector_info = sector_info[0]
    
    industry_split = item.split("Industry\n")
    industry_info = industry_split[1]
    industry_info = industry_info.split("\n")
    industry_info = industry_info[0]
    
    nominal_split = item.split("Nominal\n")
    nominal_info = nominal_split[2]
    nominal_info = nominal_info.split("\n")
    nominal_info = nominal_info[0]
    
    stocks_split = item.split("\n*** EUR")
    stock_info = stocks_split[1]
    stock_info = stock_info.replace("\n", "")
    
    relationship_split = item.split("Show more")
    relationship = relationship_split[0]
    relationship = relationship.split("\n")
    relationship = relationship[1]
    
    
    normal_split = item.split("\n")
    text_blurb = normal_split[2]
    
    comp_text.append(text_blurb)
    sector.append(sector_info)
    industry.append(industry_info)
    nominal.append(nominal_info)
    stocks.append(stock_info)
    relationship_status.append(relationship)

In [40]:
# create a new dataframe with the new variables that we have created
all_bond_info = pd.DataFrame(name_list, columns=["company_name"])

In [41]:
# add columns to the dataframe with the rest of the variables we created
all_bond_info["company_description"] = comp_text
all_bond_info["relationship_to_bond"] = relationship_status
all_bond_info["sector"] = sector
all_bond_info["industry"] = industry
all_bond_info["ISIN"] = isin_code_list
all_bond_info["bond_destination"] = bond_location
all_bond_info["coupon_rate"] = percentage_list
all_bond_info["total_bond_amount"] = amount_list
all_bond_info["nominal_amount"] = nominal
all_bond_info["quarter_created"] = cbonds_info_df["Quarter"]
all_bond_info["year_created"] = cbonds_info_df["Year"]
all_bond_info["maturity_date"] = date_list
all_bond_info["bond_currency"] = currency_list
all_bond_info["country"] = country_of_risk_list
all_bond_info["status"] = status_list
all_bond_info["stocks_info"] = stocks
all_bond_info["bond_types"] = b_types

In [42]:
# takes the different types of bond_types and provides a yes/no in seperate columns
bond_type_df = all_bond_info['bond_types'].str.get_dummies(', ').replace({0:'no', 1:'yes'})

In [43]:
all_bond_df = pd.concat([all_bond_info, bond_type_df], axis=1)

In [44]:
all_bond_df.head()

,company_name,company_description,relationship_to_bond,sector,industry,ISIN,bond_destination,coupon_rate,total_bond_amount,nominal_amount,...,status,stocks_info,bond_types,Floating rate,Foreign bonds,Green bonds,Securitization,Senior Secured,Senior Unsecured,Zero-coupon bonds
0,2i rete gas,2i rete gas,Borrower,corporate,Public utilities,XS1088274169,International bonds,1.75%,"362,793,000 EUR","1,000 EUR",...,Matured,"Par amount, integral multiple",NaN,no,no,no,no,no,no,no
1,2i rete gas,2i rete gas,Borrower,corporate,Public utilities,XS1088274672,International bonds,3%,"600,000,000 EUR","1,000 EUR",...,Outstanding,"Par amount, integral multiple",Senior Unsecured,no,no,no,no,no,yes,no
2,2i rete gas,2i rete gas,Borrower,corporate,Public utilities,XS1144492532,International bonds,1.125%,"267,100,000 EUR","1,000 EUR",...,Matured,"Par amount, integral multiple",NaN,no,no,no,no,no,no,no
3,2i rete gas,2i rete gas,Borrower,corporate,Public utilities,XS1571982468,International bonds,1.75%,"435,000,000 EUR","1,000 EUR",...,Outstanding,"Par amount, integral multiple",Senior Unsecured,no,no,no,no,no,yes,no
4,A2A,"A2A S.p.A., together with its subsidiaries, en...",Profile,corporate,Power,XS0859920406,International bonds,4.5%,"510,703,000 EUR","1,000 EUR",...,Matured,"Par amount, integral multiple",NaN,no,no,no,no,no,no,no


# Re-running for the missing items

### We realized there were 10 or so ISIN codes that didn't get grabbed, so we found which ones were skipped and reran the scraping code only for the missing ones.

In [53]:
# import missing isin list (only for debugging so the entire list doesn't have to be run again)
# path = r'/Users/gabbyvinco/Desktop/missing_isin_codes.csv'
# missing_isin = pd.read_csv(path, index_col=None, header=0)

In [54]:
# change df to list with missing isin code values
take_2 = missing_isin["ISIN_Code"].tolist()

In [61]:
take_2

['FR0012315133',
 'XS1732400319',
 'XS1759603761',
 'XS1775786574',
 'FR0013374881',
 'XS2150006562',
 'FR0013517711',
 'FR0013422227',
 'FR0014000394',
 'FR0014000C08',
 'FR0014000C73']

In [55]:
# Install and initiate driver again
driver = webdriver.Chrome(ChromeDriverManager().install())
# initiate the driver
driver.get(cbonds_url)
# initiate the action object
action = webdriver.ActionChains(driver)
# create a wait with a timeout of 30 seconds
wait = WebDriverWait(driver, 30)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [/Users/gabbyvinco/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


In [59]:
# create lists to append items from the function into 
bond_titles = []
bond_types = []
bond_amounts = []
issue_info_list = []

In [60]:
scrape_correct_names(take_2)

all info grabbed for this bond
all info grabbed for this bond
all info grabbed for this bond
all info grabbed for this bond
all info grabbed for this bond
all info grabbed for this bond
all info grabbed for this bond
all info grabbed for this bond
all info grabbed for this bond
done


In [62]:
missing_list = [1004, 1095, 1122, 1124, 1301, 1604, 1809, 1910, 1911, 1940, 1941]

In [63]:
# creating variables for the missing items 
isin_code = cbonds.iloc[missing_list, 2].tolist()
quarter = cbonds.iloc[missing_list, 0].tolist()
year = cbonds.iloc[missing_list, 1].tolist()
print(isin_code)
print(quarter)

['XS1656123459', 'FR0013292968', 'XS1764062326', 'FR0011541978', 'XS1891195775', 'XS2150015555', 'FR0013518297', 'FR0014000394', 'FR00140003A5', 'FR0014000C73', 'FR0014000C81']
['Q3', 'Q4', 'Q1', 'Q1', 'Q4', 'Q2', 'Q2', 'Q4', 'Q4', 'Q4', 'Q4']


In [64]:
# putting the missing items into its own df
missing_from_main = pd.DataFrame(bond_titles)
missing_from_main['bond_types'] = bond_types
missing_from_main['bond_amounts'] = bond_amounts
missing_from_main['issue_info'] = issue_info_list
missing_from_main['ISIN_Code'] = isin_code
missing_from_main['quarter'] = quarter
missing_from_main['year'] = year

In [65]:
# keep only missing items that we were able to get information for
missing_from_main = missing_from_main[missing_from_main.bond_types != "isin_code_not_found"]
missing_from_main

,0,bond_types,bond_amounts,issue_info,ISIN_Code,quarter,year
0,Domestic bonds: LVMH Moet Hennessy Louis Vuitt...,Floating rate,"STATUS\nMatured\nAMOUNT\n300,000,000 EUR\nPLAC...",Issue information\nProfile\nLVMH Moët Hennessy...,XS1656123459,Q3,2017
1,"International bonds: Ferrovie dello Stato, 0.8...","Green bonds, Senior Unsecured","STATUS\nOutstanding\nAMOUNT\n600,000,000 EUR\n...",Issue information\nProfile\nFerrovie dello Sta...,FR0013292968,Q4,2017
2,International bonds: Prosegur Compania de Segu...,Senior Unsecured,"STATUS\nOutstanding\nAMOUNT\n700,000,000 EUR\n...",Issue information\nBorrower\nProsegur Compania...,XS1764062326,Q1,2018
3,"International bonds: Sampo Group, 1.625% 21feb...",Senior Unsecured,"STATUS\nOutstanding\nAMOUNT\n500,000,000 EUR\n...",Issue information\nBorrower\nSampo Group\nFull...,FR0011541978,Q1,2018
4,"Domestic bonds: JCDecaux, FRN 24oct2020, EUR\n...",Floating rate,"STATUS\nMatured\nAMOUNT\n300,000,000 EUR\nPLAC...",Issue information\nProfile\nGroup profile JCDe...,XS1891195775,Q4,2018
5,"International bonds: John Deere Capital, 2.2% ...",Senior Unsecured,"STATUS\nOutstanding\nAMOUNT\n600,000,000 EUR\n...",Issue information\nProfile\nJohn Deere Capital...,XS2150015555,Q2,2020
6,"Domestic bonds: Schneider Electric, 0% 12jun20...",Zero-coupon bonds,"STATUS\nOutstanding\nAMOUNT\n500,000,000 EUR\n...",Issue information\nProfile\nSchneider Electric...,FR0013518297,Q2,2020
7,"Domestic bonds: Gecina, 1.625% 29may2034, EUR\...",,"STATUS\nOutstanding\nAMOUNT\n700,000,000 EUR\n...",Issue information\nProfile\nGecina is a reside...,FR0014000394,Q4,2020
9,"Domestic bonds: SNCF, 0.875% 28feb2051, EUR (8...",,"STATUS\nOutstanding\nAMOUNT\n2,000,000,000 EUR...",Issue information\nProfile\nSociete nationale ...,FR0014000C73,Q4,2020


In [729]:
# save the unique_improved to csv file
# missing_from_main.to_csv (r'/Users/gabbyvinco/Desktop/missingfrommain.csv', index = False, header=True)

In [66]:
# update lists so they can be added to the dataframe later
isin_code = missing_from_main["ISIN_Code"].tolist()
quarter = missing_from_main["quarter"].to_list()
year = missing_from_main["year"].tolist()

In [67]:
# exit from the driver
driver.quit()

In [68]:
# same things but for missing df

b_types_2 = missing_from_main["bond_types"]
b_types_2 = b_types_2.tolist()
print(len(b_types_2))

# change bond_amounts column to list
extra_info_2 = missing_from_main["bond_amounts"]
extra_info_2 = extra_info_2.tolist()

# change the first column to list
basic_info_2 = missing_from_main[0]
basic_info_2 = basic_info_2.tolist()

# change issue_info to list
issue_info_2 = missing_from_main['issue_info']
issue_info_2 = issue_info_2.tolist()


9


In [69]:
# create new lists to split items into from extra_info
amount_list_2 = []
status_list_2 = []
country_of_risk_list_2 = []

In [70]:
# split up the items of the bond_amounts column
for item in extra_info_2:
    item = item.split("\n")
#     print(len(item))
    amount = item[3]
    status = item[1]
    country_of_risk = item[9]
    amount_list_2.append(amount)
    status_list_2.append(status)
    country_of_risk_list_2.append(country_of_risk)

In [71]:
# create lists to split items into from basic_info
bond_location_list_2 = []
name_list_2 = []
percentage_list_2 = []
date_list_2 = []
currency_list_2 = []
isin_code_list_2 = []

In [73]:
# split up the data from the first column into its individual variables
for item in basic_info_2:
    item = item.split(",")
    bondNname = item[0]
    percNdate = item[1]
    if len(item) == 3:
        ctryNcode = item[2]
        ctryNcode = ctryNcode.split("\n")
        currency = ctryNcode[0]
    #     print(currency)
        isin_code = ctryNcode[1]
    else:
        isin_code = "none"
        currency = "none"
    currency_list_2.append(currency)
    isin_code_list_2.append(isin_code)
        
    bondNname = bondNname.split(": ")
    bond_location = bondNname[0]
    name = bondNname[1]
    bond_location_list_2.append(bond_location)
    name_list_2.append(name)
    
    percNdate = percNdate.split(" ")
    if len(percNdate) == 2:
        sub_item = percNdate[1]
        sub_item = sub_item.split("%")
        date = sub_item[1]
        percentage = sub_item[0]+"%"
        percentage_list_2.append(percentage)
        date_list_2.append(date)
    else:
        percentage = percNdate[1]
        date = percNdate[2]
        percentage_list_2.append(percentage)
        date_list_2.append(date)
    
    
    

In [74]:
# create lists to split items into from issue_info
comp_text_2 = []
sector_2 = []
industry_2 = []
nominal_2 = []
stocks_2 = []
relationship_status_2 = []

In [76]:
for item in issue_info_2:
    sector_split = item.split("Sector\n")
    sector_info = sector_split[1]
    sector_info = sector_info.split("\n")
    sector_info = sector_info[0]
    
    industry_split = item.split("Industry\n")
    industry_info = industry_split[1]
    industry_info = industry_info.split("\n")
    industry_info = industry_info[0]
    
    nominal_split = item.split("Nominal\n")
    nominal_info = nominal_split[2]
    nominal_info = nominal_info.split("\n")
    nominal_info = nominal_info[0]
#     print(nominal_info)
    
    stocks_split = item.split("\n*** EUR")
    stock_info = stocks_split[1]
    stock_info = stock_info.replace("\n", "")
    
    relationship_split = item.split("Show more")
    relationship = relationship_split[0]
    relationship = relationship.split("\n")
    relationship = relationship[1]
    
    
    normal_split = item.split("\n")
    text_blurb = normal_split[2]
    
    comp_text_2.append(text_blurb)
    sector_2.append(sector_info)
    industry_2.append(industry_info)
    nominal_2.append(nominal_info)
    stocks_2.append(stock_info)
    relationship_status_2.append(relationship)

In [77]:
missing_bond_info = pd.DataFrame(name_list_2, columns=["company_name"])

In [79]:
missing_bond_info["company_description"] = comp_text_2
missing_bond_info["relationship_to_bond"] = relationship_status_2
missing_bond_info["sector"] = sector_2
missing_bond_info["industry"] = industry_2
missing_bond_info["ISIN"] = isin_code
missing_bond_info["bond_destination"] = bond_location_list_2
missing_bond_info["coupon_rate"] = percentage_list_2
missing_bond_info["total_bond_amount"] = amount_list_2
missing_bond_info["nominal_amount"] = nominal_2
missing_bond_info["quarter_created"] = quarter
missing_bond_info["year_created"] = year
missing_bond_info["maturity_date"] = date_list_2
missing_bond_info["bond_currency"] = currency_list_2
missing_bond_info["country"] = country_of_risk_list_2
missing_bond_info["status"] = status_list_2
missing_bond_info["stocks_info"] = stocks_2
missing_bond_info["bond_types"] = b_types_2

In [80]:
# takes the different types of bond_types and provides a yes/no in seperate columns
missing_bond_type_df = missing_bond_info['bond_types'].str.get_dummies(', ').replace({0:'no', 1:'yes'})

In [81]:
missing_bonds_df = pd.concat([missing_bond_info, missing_bond_type_df], axis=1)

In [82]:
missing_bonds_df

,company_name,company_description,relationship_to_bond,sector,industry,ISIN,bond_destination,coupon_rate,total_bond_amount,nominal_amount,...,maturity_date,bond_currency,country,status,stocks_info,bond_types,Floating rate,Green bonds,Senior Unsecured,Zero-coupon bonds
0,LVMH Moet Hennessy Louis Vuitton,LVMH Moët Hennessy - Louis Vuitton SA engages ...,Profile,corporate,Light industry,FR0014000C08,Domestic bonds,FRN,"300,000,000 EUR","100,000 EUR",...,14feb2019,EUR,France,Matured,"Par amount, integral multiple",Floating rate,yes,no,no,no
1,Ferrovie dello Stato,"Ferrovie dello Stato Italiane SpA, together wi...",Profile,corporate,Transportation,FR0014000C08,International bonds,0.875%,"600,000,000 EUR","1,000 EUR",...,7dec2023,EUR,Italy,Outstanding,"Par amount, integral multiple","Green bonds, Senior Unsecured",no,yes,yes,no
2,Prosegur Compania de Seguridad SA,Prosegur Compania de Seguridad SA,Borrower,corporate,Financial institutions,FR0014000C08,International bonds,1%,"700,000,000 EUR","100,000 EUR",...,8feb2023,EUR,Spain,Outstanding,"Par amount, integral multiple",Senior Unsecured,no,no,yes,no
3,Sampo Group,Sampo Group,Borrower,corporate,Financial institutions,FR0014000C08,International bonds,1.625%,"500,000,000 EUR","1,000 EUR",...,21feb2028,EUR,Finland,Outstanding,"Par amount, integral multiple",Senior Unsecured,no,no,yes,no
4,JCDecaux,Group profile JCDecaux SA is the number one ou...,Profile,corporate,Other sectors,FR0014000C08,Domestic bonds,FRN,"300,000,000 EUR","100,000 EUR",...,24oct2020,EUR,France,Matured,"Par amount, integral multiple",Floating rate,yes,no,no,no
5,John Deere Capital,John Deere Capital Corporation provides machin...,Profile,corporate,Financial institutions,FR0014000C08,International bonds,2.2%,"600,000,000 EUR","100,000 EUR",...,2apr2032,EUR,USA,Outstanding,"Par amount, integral multiple",Senior Unsecured,no,no,yes,no
6,Schneider Electric,Schneider Electric SA specializes in energy ma...,Profile,corporate,Power,FR0014000C08,Domestic bonds,0%,"500,000,000 EUR","100,000 EUR",...,12jun2023,EUR,France,Outstanding,"Par amount, integral multiple",Zero-coupon bonds,no,no,no,yes
7,Gecina,"Gecina is a residential, office, and healthcar...",Profile,corporate,Financial institutions,FR0014000C08,Domestic bonds,1.625%,"700,000,000 EUR","100,000 EUR",...,29may2034,EUR,France,Outstanding,"Par amount, integral multiple",,no,no,no,no
8,SNCF,"Societe nationale SNCF SA (before January 1, 2...",Profile,corporate,Transportation,FR0014000C08,Domestic bonds,0.875%,"2,000,000,000 EUR","100,000 EUR",...,28feb2051,EUR (8),France,Outstanding,"Par amount, integral multiple",,no,no,no,no


# Combine main df with missing df

In [83]:
all_bond_df = all_bond_df.append(missing_bonds_df, sort=False)

In [84]:
all_bond_df["Floating rate"] = all_bond_df["Floating rate"].replace(np.nan, 'no', regex=True)
all_bond_df["Foreign bonds"] = all_bond_df["Foreign bonds"].replace(np.nan, 'no', regex=True)
all_bond_df["Green bonds"] = all_bond_df["Green bonds"].replace(np.nan, 'no', regex=True)
all_bond_df["Securitization"] = all_bond_df["Securitization"].replace(np.nan, 'no', regex=True)
all_bond_df["Senior Secured"] = all_bond_df["Senior Secured"].replace(np.nan, 'no', regex=True)
all_bond_df["Senior Unsecured"] = all_bond_df["Senior Unsecured"].replace(np.nan, 'no', regex=True)
all_bond_df["Zero-coupon bonds"] = all_bond_df["Zero-coupon bonds"].replace(np.nan, 'no', regex=True)

In [85]:
all_bond_df = all_bond_df.sort_values(["year_created", "quarter_created"], ascending = (True, True))

In [86]:
pd.set_option('display.max_columns', None)
all_bond_df.head()

,company_name,company_description,relationship_to_bond,sector,industry,ISIN,bond_destination,coupon_rate,total_bond_amount,nominal_amount,quarter_created,year_created,maturity_date,bond_currency,country,status,stocks_info,bond_types,Floating rate,Foreign bonds,Green bonds,Securitization,Senior Secured,Senior Unsecured,Zero-coupon bonds
0,2i rete gas,2i rete gas,Borrower,corporate,Public utilities,XS1088274169,International bonds,1.75%,"362,793,000 EUR","1,000 EUR",Q2,2017,16jul2019,EUR,Italy,Matured,"Par amount, integral multiple",NaN,no,no,no,no,no,no,no
1,2i rete gas,2i rete gas,Borrower,corporate,Public utilities,XS1088274672,International bonds,3%,"600,000,000 EUR","1,000 EUR",Q2,2017,16jul2024,EUR,Italy,Outstanding,"Par amount, integral multiple",Senior Unsecured,no,no,no,no,no,yes,no
2,2i rete gas,2i rete gas,Borrower,corporate,Public utilities,XS1144492532,International bonds,1.125%,"267,100,000 EUR","1,000 EUR",Q2,2017,2jan2020,EUR,Italy,Matured,"Par amount, integral multiple",NaN,no,no,no,no,no,no,no
3,2i rete gas,2i rete gas,Borrower,corporate,Public utilities,XS1571982468,International bonds,1.75%,"435,000,000 EUR","1,000 EUR",Q2,2017,28aug2026,EUR,Italy,Outstanding,"Par amount, integral multiple",Senior Unsecured,no,no,no,no,no,yes,no
4,A2A,"A2A S.p.A., together with its subsidiaries, en...",Profile,corporate,Power,XS0859920406,International bonds,4.5%,"510,703,000 EUR","1,000 EUR",Q2,2017,28nov2019,EUR,Italy,Matured,"Par amount, integral multiple",NaN,no,no,no,no,no,no,no


In [87]:
len(all_bond_df)

2083

In [857]:
# save the unique_improved to csv file
# all_bond_df.to_csv (r'/Users/gabbyvinco/Desktop/HISTORICAL_BONDS.csv', index = False, header=True)